In [2]:
import pandas as pd
import os
import re

# 1. Fonction de nettoyage spécifique pour les Programmes (Platforms)
def clean_platform_text(text):
    # Supprimer les mentions légales typiques des PDF de campagne
    text = re.sub(r'Paid for by.*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Not Authorized By.*', '', text, flags=re.IGNORECASE)
    
    # Supprimer les numéros de page isolés (chiffres seuls sur une ligne ou entourés d'espaces)
    text = re.sub(r'\n\s*\d+\s*\n', ' ', text) # Chiffre seul sur une ligne
    text = re.sub(r'\s\d+\s', ' ', text)       # Chiffre isolé dans le texte
    
    # Supprimer les en-têtes/pieds de page répétés s'ils sont connus (ex: "Republican Platform 2012")
    # On peut ajouter ici des patterns spécifiques si tu en remarques d'autres
    
    # Nettoyage des sauts de ligne (PDF) : 
    # Remplacer les retours à la ligne par des espaces pour reconstituer les phrases
    text = text.replace('\n', ' ')
    
    # Supprimer les espaces multiples (ex: "  " devient " ")
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# 2. Boucle pour traiter les fichiers
def process_platforms(folder_path):
    data = []
    
    # Liste tous les fichiers .txt dans le dossier
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            
            # Extraction des métadonnées via le nom du fichier (ex: "2004-Republicans.txt")
            # Attention: Adapte le split si tes fichiers sont nommés différemment
            try:
                year_str, party_raw = filename.replace('.txt', '').split('-')
                year = int(year_str)
                party = "Republican" if "Republicans" in party_raw else "Democrat"
            except:
                print(f"Format de nom de fichier non reconnu pour : {filename}")
                continue

            # Lecture et nettoyage
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                raw_text = f.read()
                cleaned_text = clean_platform_text(raw_text)
            
            # Pour les programmes, le candidat est souvent "Party Platform", 
            # ou tu peux mettre le nom du candidat de l'année si tu préfères.
            candidate_name = "Party Platform" 
            
            data.append({
                'year': year,
                'party': party,
                'candidate': candidate_name, 
                'text': cleaned_text
            })
    
    return pd.DataFrame(data)

# --- UTILISATION ---

# Remplace ceci par le chemin réel de ton dossier contenant les 14 fichiers txt
folder_path = '/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/raw/manifestos' 

# Création du dataframe des programmes
df_platforms = process_platforms(folder_path)

# Chargement de ton dataframe des discours (celui que tu as envoyé)
df_speeches = pd.read_csv('/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/president_speeches_clean.csv')

# Uniformisation des colonnes (au cas où)
df_speeches = df_speeches[['year', 'party', 'candidate', 'text']]
df_platforms = df_platforms[['year', 'party', 'candidate', 'text']]

# Fusion des deux bases (Concaténation verticale)
df_nlp_final = pd.concat([df_speeches, df_platforms], ignore_index=True)

# Vérification
print(f"Nombre de textes total : {len(df_nlp_final)}")
print(df_nlp_final.head())

# Sauvegarde
df_nlp_final.to_csv('nlp_database_complete.csv', index=False)

Format de nom de fichier non reconnu pour : 2020_Republicans.txt
Nombre de textes total : 27
   year       party candidate  \
0  2024    Democrat    Harris   
1  2024  Republican     Trump   
2  2020    Democrat     Biden   
3  2020  Republican     Trump   
4  2016    Democrat   Clinton   

                                                text  
0  Good evening! Kamala! Kamala! Kamala! Californ...  
1  Thank you very much. Thank you very, very much...  
2  Good evening. Ella Baker, a giant of the civil...  
3  Thank you very much. Thank you very much. Than...  
4  Thank you all very, very much! Thank you for t...  


Sanity check

In [4]:
# 1. REMETTRE DE L'ORDRE
# On trie par Année (descendant) et par Parti
df_nlp_final = df_nlp_final.sort_values(by=['year', 'party'], ascending=[False, True])

# 2. VÉRIFICATION (Le "Sanity Check")
print("-" * 30)
print("VÉRIFICATION DU CONTENU")
print("-" * 30)

# A. Est-ce qu'on a bien tous les documents ?
# Tu devrais avoir environ 4 documents par an (2 discours + 2 programmes)
print("\nNombre de documents par année :")
print(df_nlp_final.groupby('year').size())

# B. Est-ce que les textes des programmes sont bien remplis ?
# On calcule la longueur du texte pour vérifier qu'on n'a pas nettoyé "trop fort" (texte vide)
df_nlp_final['text_length'] = df_nlp_final['text'].apply(len)

print("\nAperçu des 10 premiers documents (Année, Parti, Candidat, Longueur du texte) :")
print(df_nlp_final[['year', 'party', 'candidate', 'text_length']].head(15))

# C. Vérifier spécifiquement les plateformes (Programmes)
print("\nZoom sur les programmes (doivent être très longs, > 20 000 caractères souvent) :")
print(df_nlp_final[df_nlp_final['candidate'] == "Party Platform"][['year', 'party', 'text_length']])

------------------------------
VÉRIFICATION DU CONTENU
------------------------------

Nombre de documents par année :
year
2000    4
2004    4
2008    4
2012    4
2016    4
2020    3
2024    4
dtype: int64

Aperçu des 10 premiers documents (Année, Parti, Candidat, Longueur du texte) :
    year       party       candidate  text_length
0   2024    Democrat          Harris        20974
20  2024    Democrat  Party Platform       270996
1   2024  Republican           Trump        66964
22  2024  Republican  Party Platform        39275
2   2020    Democrat           Biden        17591
19  2020    Democrat  Party Platform       287421
3   2020  Republican           Trump        40805
4   2016    Democrat         Clinton        29502
15  2016    Democrat  Party Platform       182915
5   2016  Republican           Trump        29128
23  2016  Republican  Party Platform       240697
6   2012    Democrat           Obama        25330
14  2012    Democrat  Party Platform       169538
7   2012  Rep

In [5]:
# On regarde uniquement l'année 2012
test_2012 = df_nlp_final[df_nlp_final['year'] == 2012]

print(f"Nombre de lignes pour 2012 : {len(test_2012)}")
print(test_2012[['year', 'party', 'candidate', 'text_length']])

Nombre de lignes pour 2012 : 4
    year       party       candidate  text_length
6   2012    Democrat           Obama        25330
14  2012    Democrat  Party Platform       169538
7   2012  Republican          Romney        22519
16  2012  Republican  Party Platform       211417


tri dans la base de donnée 

In [7]:
# --- ÉTAPE 1 : TRI DÉFINITIF ET PROPRE ---

# 1. On trie par Année (descendant) et par Parti (alphabétique)
df_nlp_final = df_nlp_final.sort_values(by=['year', 'party'], ascending=[False, True])

# 2. CRUCIAL : On réinitialise l'index
# Cela permet que la première ligne soit bien la numéro 0, la deuxième la 1, etc.
# L'option 'drop=True' évite de garder l'ancien index en désordre dans une nouvelle colonne.
df_nlp_final = df_nlp_final.reset_index(drop=True)

# 3. On sauvegarde cette version "propre et triée" (checkpoint de sécurité)
df_nlp_final.to_csv('nlp_database_sorted_clean.csv', index=False)

print("✅ Base de données triée et index réinitialisé.")
print("Aperçu des 10 premières lignes (doit commencer par 2024) :")
print(df_nlp_final[['year', 'party', 'candidate']].head(10))

✅ Base de données triée et index réinitialisé.
Aperçu des 10 premières lignes (doit commencer par 2024) :
   year       party       candidate
0  2024    Democrat          Harris
1  2024    Democrat  Party Platform
2  2024  Republican           Trump
3  2024  Republican  Party Platform
4  2020    Democrat           Biden
5  2020    Democrat  Party Platform
6  2020  Republican           Trump
7  2016    Democrat         Clinton
8  2016    Democrat  Party Platform
9  2016  Republican           Trump


In [21]:
import pandas as pd
import re

def cleaning_v8_scorched_earth(text):
    if not isinstance(text, str): return ""
    
    # --- 1. NETTOYAGE DES NOMBRES (NOUVEAU) ---
    # On supprime TOUS les nombres qui ne ressemblent pas à une année (19xx ou 20xx)
    # Cela va tuer tes "6060", "0002", "3-1", etc.
    # Regex : On cherche un chiffre. Si ce n'est pas précédé de 19 ou 20, on supprime.
    text = re.sub(r'\b(?!19|20)\d+\b', ' ', text) # Vire 6060, 54, 1...
    text = re.sub(r'\b\d+\.\d+\b', ' ', text)     # Vire les décimaux 0.0002
    
    # --- 2. LA BLACKLIST (Mots individuels) ---
    # On ajoute ici tous les déchets que tu as vus, mot par mot.
    blacklist = [
        "etek", "nee", "ane", "EERE", "PR", "ees", "MERICAN", 
        "itecee", "cscce", "erenere", "ri", "DDLE", "eeaeva"
    ]
    
    for bad_word in blacklist:
        # On supprime le mot exact, quelle que soit la casse
        text = re.sub(r'\b' + re.escape(bad_word) + r'\b', ' ', text, flags=re.IGNORECASE)

    # --- 3. NETTOYAGE PONCTUATION & SYMBOLES (NOUVEAU) ---
    # On vire les répétitions de ponctuation (ex: ": :..")
    text = re.sub(r'[\.:\-,_]{2,}', ' ', text) 
    # On vire le symbole centime et autres
    text = re.sub(r'[^a-zA-Z0-9\s.,;:\'\-?!]', ' ', text)

    # --- 4. RÈGLES CLASSIQUES (Déjà validées) ---
    # Noise Pattern (Lettres sales OCR) -> Mots > 12 lettres
    text = re.sub(r'\b[cesinrat_\-]{12,}\b', ' ', text, flags=re.IGNORECASE)
    
    # Length Nuke (Mots > 18 lettres)
    text = re.sub(r'\b\w{18,}\b', ' ', text)
    
    # Majuscules espacées
    text = re.sub(r'\b([A-Z])\s+(?=[A-Z]\b)', r'\1', text)
    
    # En-têtes
    text = re.sub(r'TABLE OF CONTENTS|CONTENTS|PREAMBLE|INTRODUCTION', ' ', text, flags=re.IGNORECASE)

    # Nettoyage final des espaces
    return re.sub(r'\s+', ' ', text).strip()

print("☢️ Démarrage du nettoyage V8 (Terre Brûlée)...")

# Application
df_nlp_final['text'] = df_nlp_final['text'].apply(cleaning_v8_scorched_earth)

# Tri et Indexation
df_nlp_final = df_nlp_final.sort_values(by=['year', 'party'], ascending=[False, True])
df_nlp_final = df_nlp_final.reset_index(drop=True)

# Sauvegarde
output_filename = 'nlp_database_CLEAN_V8.csv'
df_nlp_final.to_csv(output_filename, index=False)

print(f"✅ Terminé ! Base sauvegardée : {output_filename}")

# --- VÉRIFICATION ---
print("\n🔍 Vérification finale :")
check_list = ["etek", "nee", "6060", "0002", "ees", "MERICAN"]
clean_count = 0

for item in check_list:
    matches = df_nlp_final[df_nlp_final['text'].str.contains(r'\b' + item + r'\b', case=False, na=False)]
    if len(matches) > 0:
        print(f"❌ '{item}' est toujours là (Bizarre !)")
    else:
        print(f"✨ '{item}' a été atomisé.")
        clean_count += 1

if clean_count == len(check_list):
    print("\n🏆 VICTOIRE TOTALE. Lance l'analyse de sentiment maintenant !")

☢️ Démarrage du nettoyage V8 (Terre Brûlée)...
✅ Terminé ! Base sauvegardée : nlp_database_CLEAN_V8.csv

🔍 Vérification finale :
✨ 'etek' a été atomisé.
✨ 'nee' a été atomisé.
✨ '6060' a été atomisé.
✨ '0002' a été atomisé.
✨ 'ees' a été atomisé.
✨ 'MERICAN' a été atomisé.

🏆 VICTOIRE TOTALE. Lance l'analyse de sentiment maintenant !
